<span style="color: red;"><i>Only run this if the file "player_stats.csv" exists.</i></span>

# Continue Scraping
### This file should be run when you want to pickup where you left off scraping

In [7]:
# Dependencies
%pip install pandas
%pip install -U scikit-learn
%pip install lxml
%pip install html5lib
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import random
import time
from IPython.display import clear_output

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
# figure what player and what year we left off at
player_list = pd.read_csv('players_list.csv')
num_players = len(player_list)
player_stats = pd.read_csv('player_stats.csv')
last_row = player_stats.tail(1)
last_player_name = last_row["Name"].values[0]
last_player_year = int(last_row["Date"].values[0][:4])
print(last_player_name, last_player_year, last_row)

Ochai Agbaji 2024       Rk   G        Date     Age   Tm Unnamed: 5  Opp Unnamed: 7 GS     MP  \
1086  83  78  2024-04-14  23-360  TOR          @  MIA    L (-15)  1  32:45   

      ... TRB AST STL BLK TOV PF PTS GmSc +/-          Name  
1086  ...   6   2   2   1   4  3  10  5.8  +1  Ochai Agbaji  

[1 rows x 31 columns]


In [9]:
# get player from list of players see if we got all of his years
last_player_row = player_list.loc[player_list['Player'] == last_player_name]
player_index = last_player_row.index[0]
player_last_year = last_player_row["To"].values[0]
# if this is true start from next player
player_recorded = last_player_year == player_last_year

### Helper Functions

In [10]:
# continue scraping from where left off
def get_gamelog_year_url(row, year):
    player_name = row['Player'].lower().split()
    
    first_letter_last = player_name[1][0]
    unique_id = player_name[1][:5] + player_name[0][:2]
    return f"https://www.basketball-reference.com/players/{first_letter_last}/{unique_id}01/gamelog/{year}"

# returns an array of the years that the player played in the NBA
def get_player_years_played(player_row):
    return list(range(player_row["From"], player_row["To"] + 1))


def storePlayerYearData(row, year_played):
    url = get_gamelog_year_url(row, year_played)
    time.sleep(random.randint(2, 5))
    print(url)
    tables = pd.read_html(url)
    player_year_data = tables[7]
    if player_year_data is not None and not player_year_data.empty: # this could be undefined, might have played in the g-league
        player_year_data["Name"] = row["Player"]
        player_stats = pd.read_csv('player_stats.csv')
        player_stats = pd.concat([player_stats, player_year_data], ignore_index=True)
        player_stats.to_csv('player_stats.csv', index=False)


In [11]:
if(player_recorded == False):
    # finish up with that player
    print(last_player_year, last_player_row["To"].values[0] + 1)
    for year_played in list(range(last_player_year + 1, last_player_row["To"].values[0] + 1)):
        storePlayerYearData(last_player_row.squeeze(), year_played)

In [12]:
print(f"continuing to proccess player data for {num_players - player_index} players")
# itterates over every player
for index in range(player_index + 1, num_players - 1): # change this for production
    print(f"Player {index} of {num_players} players")
    # this is the player
    row = player_list.iloc[index]
    years_played = get_player_years_played(row)
    for year_played in years_played:
        print(f"Year {year_played}")
        storePlayerYearData(row, year_played)
    clear_output(wait=True)

Player 11 of 572 players
Year 2016
https://www.basketball-reference.com/players/a/antetth01/gamelog/2016
Year 2017
https://www.basketball-reference.com/players/a/antetth01/gamelog/2017


ValueError: No tables found